Finding congressional vulnerability by district for advocacy efforts. Combined data from Yale Climate Opinion Map, LCV (extracting data using BeautifulSoup), cleaning data with Stephen Wolf's Congressional dataset, and creating a simple correlation formula comparing amount of concern about climate change to a congressperson's voting record.

In [183]:
import pandas as pd 
import sqlite3
import numpy as np

survey = pd.read_csv('YCOM_2020_Data.csv')
survey.head()
survey.groupby(['GeoType']).count()
worried = survey.sort_values(by=['worried'], ascending=False)
worried.head(20)
cd= worried[worried['GeoType']=='cd116']
cd.sort_values(by=['GEOID'], inplace=True)
cd['GeoName'].to_csv('cdnames.csv', index=False)
party = pd.read_csv('party_affiliation.csv')
cd = cd.merge(party, how ='inner', on='GEOID')
cd_names = pd.read_csv('cd_names1.csv')
cd = cd.merge(cd_names, how ='inner', on='GEOID')
cd
cd = cd[['GEOID','cd_name', 'worried', 'Party']]
cd
lcv= pd.read_csv('lcv1.csv')
cd = cd.merge(lcv, how ='inner', on='GEOID')
cd= cd.sort_values(by='cd_name')
cd
cd.astype({'lifetime_rating': 'float', '2019_rating': 'float'}).dtypes
cd['discrepancy_lifetime']= cd['worried']-cd['lifetime_rating']
cd['discrepancy_2019']= cd['worried']-cd['2019_rating']
cd = cd.sort_values(by='discrepancy_2019', ascending=False)
cd.to_csv('congressional_climate_vulnerability.csv')

cd.head(50)


#cd = cd.loc[(cd.worried > 60) & (cd.Party == 'Republican')]
#worried_republicans = cd.sort_values (by=['worried'], ascending = False)
#worried_republicans
#worried_republicans.to_csv('worried_republican_districts.csv')
#pd.concat([cd[[0]], cd[1].str.split(', ', expand=True)], axis=1)

<ipython-input-183-e76b9d9e6d80>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cd.sort_values(by=['GEOID'], inplace=True)


,GEOID,cd_name,worried,Party,2019_rating,lifetime_rating,name,discrepancy_lifetime,discrepancy_2019
42,622,California's 22nd,66.208,Republican,0,3,"Nunes, Devin G.",63.208,66.208
386,4825,Texas' 25th,63.741,Republican,0,1,"Williams, Roger",62.741,63.741
382,4821,Texas' 21st,63.402,Republican,0,0,"Roy, Chip",63.402,63.402
388,4827,Texas' 27th,62.991,Republican,0,0,"Cloud, Michael",62.991,62.991
392,4831,Texas' 31st,64.371,Republican,3,4,"Carter, John R.",60.371,61.371
334,4210,Pennsylvania's 10th,63.210,Republican,3,3,"Perry, Scott",60.210,60.210
293,3709,North Carolina's 9th,59.941,Republican,0,0,"Bishop, Dan",59.941,59.941
12,405,Arizona's 5th,59.633,Republican,0,6,"Biggs, Andy",53.633,59.633
385,4824,Texas' 24th,62.374,Republican,3,3,"Marchant, Kenny E.",59.374,59.374
24,604,California's 4th,61.879,Republican,3,4,"McClintock, Tom",57.879,58.879


learning beautiful soup to extract scorecard dataset from LCV 

In [158]:
import requests
from bs4 import BeautifulSoup

result = requests.get("https://scorecard.lcv.org/members-of-congress")
src=result.content
soup = BeautifulSoup(src)
table = soup.find_all('span',{'class': "mocState"})
names = []
table
for state in table:
    names.append(state.get('sort'))
names

df = pd.DataFrame()
df['state']=names
df


table2 = soup.find_all("span", class_="mocLifetimeRating")
table2
table2
lr = []
for LR in table2:
    x= LR.get('sort')
    lr.append(x)
lr
df['lifetime_rating']=lr
df

table3 = soup.find_all("span", class_="mocRating")

thisyear = []
for LR in table3:
    x= LR.get('sort')
    thisyear.append(x)
thisyear
df['2019_rating']= thisyear
df

table4 = soup.find_all("span", class_="mocName")

name = []
for LR in table4:
    x= LR.get('sort')
    name.append(x)
thisyear
df['name']= name
df

table5 = soup.find_all("span", class_="mocParty")

party = []
for LR in table5:
    x= LR.get('sort')
    party.append(x)
df['party']= party
df.sort_values(by=['state'])
df['state'].tolist()
df = df[101:]
df = df.sort_values(by='state', ascending=True)
df.to_csv('lcv_scores.csv')
df

#for a_tag in links:
#    urls.append(a_tag.attrs['href'])
#print(urls)

,state,lifetime_rating,2019_rating,name,party
131,Alabama-01,000,003,"Byrne, Bradley",R
165,Alabama-02,004,003,"Roby, Martha",R
225,Alabama-03,005,003,"Rogers, Mike D.",R
256,Alabama-04,003,003,"Aderholt, Robert B.",R
292,Alabama-05,008,003,"Brooks, Mo",R
...,...,...,...,...,...
285,Wisconsin-05,024,000,"Sensenbrenner, James F.",R
311,Wisconsin-06,002,010,"Grothman, Glenn",R
328,Wisconsin-07,002,005,"Duffy, Sean",R
364,Wisconsin-08,005,014,"Gallagher, Mike",R


In [ ]:
import pandas as pd
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)")
dfs[5]